<a href="https://colab.research.google.com/github/praveenjune17/BERT_text_summarisation/blob/master/Text_summarization_demo_using_BERT_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Text summarization using BERT.**




---



########################################################################################################################
## Instructions for running the script
  a) Select  Open in Google colab 
  b) Select  Runtime. 'change runtime type' to GPU
      c) Select --> Run all (top left corner)
  
  d) Takes few mins to load the dependencies in the end you will be prompted to enter a Document. Summary will be created for this
########################################################################################################################

In [0]:
!git clone https://github.com/praveenjune17/BERT_text_summarisation
!pip install bunch
!pip install rouge==0.3.2
!pip install bert_score 
!pip install --upgrade grpcio
!pip install tensorflow_addons

In [1]:
import tensorflow as tf
import time
import shutil
import os
from google_drive_downloader import GoogleDriveDownloader as gdd
import sys
sys.path.insert(0, '/content/BERT_text_summarisation/scripts')
from metrics import convert_wordpiece_to_words
from decode_text import *

setting default value to last_recorded_value
INFO:tensorflow:Extracting pretrained word embeddings weights from BERT
INFO:tensorflow:Embedding matrix shape '(30522, 768)'
INFO:tensorflow:Loading Pre-trained BERT model for BERT SCORE calculation
INFO:tensorflow:Extracting pretrained word embeddings weights from BERT
INFO:tensorflow:Embedding matrix shape '(30522, 768)'


Download checkpoints from Gdrive

In [0]:
!pip install tensor2tensor

In [18]:
gdd.download_file_from_google_drive(file_id='9Xjo41IDfgaoGXjdEyBmhucK2CHpkaZ',
                                      dest_path='/root/ckpt_dir/infer.zip',
                                      unzip=True)

Unzipping...

/usr/local/lib/python3.6/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "9Xjo41IDfgaoGXjdEyBmhucK2CHpkaZ" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [22]:
def summarize(infer_ckpt):
  ckpt = tf.train.Checkpoint(
                             model=model
                            )
  ckpt.restore('/content/drive/My Drive/Text_summarization/BERT_text_summarisation/Summarization_inference_ckps/ckpt-75').expect_partial()
  start = time.time()
  ip_ids = tokenizer.encode(input('Enter a news article to be summarized --> '))
  preds_draft_summary, preds_refined_summary, refine_attention_dist = predict_using_beam_search(tf.convert_to_tensor([ip_ids]),
                                                                                               refine_decoder_sampling_type='greedy',
                                                                                               k=7,
                                                                                               p=1)
  sum_hyp = tokenizer.convert_ids_to_tokens([i for i in tf.squeeze(preds_refined_summary) if i not in [CLS_ID, SEP_ID, 0]])
  sum_hyp = convert_wordpiece_to_words(sum_hyp)
  print(f'the summarized output is --> {sum_hyp if sum_hyp else "EMPTY"}')
  print(f'Time to process {round(time.time() - start)} seconds')

if __name__ == '__main__':
  infer_ckpt = '75'
  summarize(infer_ckpt)  

Enter a news article to be summarized --> A wiki is run using wiki software, otherwise known as a wiki engine. A wiki engine is a type of content management system, but it differs from most other such systems, including blog software, in that the content is created without any defined owner or leader, and wikis have little inherent structure, allowing structure to emerge according to the needs of the users.[2] There are dozens of different wiki engines in use, both standalone and part of other software, such as bug tracking systems. Some wiki engines are open source, whereas others are proprietary. Some permit control over different functions (levels of access); for example, editing rights may permit changing, adding, or removing material. Others may permit access without enforcing access control. Other rules may be imposed to organize content.
INFO:tensorflow:Building: 'Draft beam search decoder'
INFO:tensorflow:Building: 'Refined greedy decoder'
the summarized output is --> wiki soft

In [0]:
#sample text

A wiki is run using wiki software, otherwise known as a wiki engine. A wiki engine is a type of content management system, but it differs from most other such systems, including blog software, in that the content is created without any defined owner or leader, and wikis have little inherent structure, allowing structure to emerge according to the needs of the users.[2] There are dozens of different wiki engines in use, both standalone and part of other software, such as bug tracking systems. Some wiki engines are open source, whereas others are proprietary. Some permit control over different functions (levels of access); for example, editing rights may permit changing, adding, or removing material. Others may permit access without enforcing access control. Other rules may be imposed to organize content.